# Building Machine Learning Classifiers: Explore Random Forest model with grid-search

**Grid-search:** Exhaustively search all parameter combinations in a given grid to determine the best model.

### Read in & clean text

In [1]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])

X_features = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)
X_features.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,49,4.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,62,3.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,28,7.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,135,4.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Build our own Grid-search

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X_features, data['label'], test_size=0.2)

In [4]:
def train_RF(n_est, depth):
    rf = RandomForestClassifier(n_estimators=n_est, max_depth=depth, n_jobs=-1)
    rf_model = rf.fit(X_train, y_train)
    y_pred = rf_model.predict(X_test)
    precision, recall, fscore, support = score(y_test, y_pred, pos_label='spam', average='binary')
    print('Est: {} / Depth: {} ---- Precision: {} / Recall: {} / Accuracy: {} / f-Score: {} / Support: {}'.format(
        n_est, 
        depth, 
        round(precision, 3), 
        round(recall, 3), 
        round((y_pred == y_test).sum() / len(y_pred), 3),
        round(fscore, 3),
        support
    ))
    
   

In [5]:
for n_est in [10, 50, 100, 150, 200]:
    for depth in [5, 10, 15, 20, 30, None]:
        train_RF(n_est, depth)

Est: 10 / Depth: 5 ---- Precision: 1.0 / Recall: 0.045 / Accuracy: 0.866 / f-Score: 0.086 / Support: None
Est: 10 / Depth: 10 ---- Precision: 1.0 / Recall: 0.173 / Accuracy: 0.884 / f-Score: 0.295 / Support: None
Est: 10 / Depth: 15 ---- Precision: 0.973 / Recall: 0.455 / Accuracy: 0.922 / f-Score: 0.62 / Support: None
Est: 10 / Depth: 20 ---- Precision: 1.0 / Recall: 0.538 / Accuracy: 0.935 / f-Score: 0.7 / Support: None
Est: 10 / Depth: 30 ---- Precision: 0.99 / Recall: 0.647 / Accuracy: 0.95 / f-Score: 0.783 / Support: None
Est: 10 / Depth: None ---- Precision: 0.977 / Recall: 0.801 / Accuracy: 0.969 / f-Score: 0.88 / Support: None
Est: 50 / Depth: 5 ---- Precision: 1.0 / Recall: 0.019 / Accuracy: 0.863 / f-Score: 0.038 / Support: None
Est: 50 / Depth: 10 ---- Precision: 1.0 / Recall: 0.231 / Accuracy: 0.892 / f-Score: 0.375 / Support: None
Est: 50 / Depth: 15 ---- Precision: 1.0 / Recall: 0.397 / Accuracy: 0.916 / f-Score: 0.569 / Support: None
Est: 50 / Depth: 20 ---- Precision: 1

/opt/miniconda3/envs/sb-demos-and-mini-projects/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Est: 200 / Depth: 5 ---- Precision: 0.0 / Recall: 0.0 / Accuracy: 0.86 / f-Score: 0.0 / Support: None
Est: 200 / Depth: 10 ---- Precision: 1.0 / Recall: 0.224 / Accuracy: 0.891 / f-Score: 0.366 / Support: None
Est: 200 / Depth: 15 ---- Precision: 1.0 / Recall: 0.429 / Accuracy: 0.92 / f-Score: 0.601 / Support: None
Est: 200 / Depth: 20 ---- Precision: 1.0 / Recall: 0.59 / Accuracy: 0.943 / f-Score: 0.742 / Support: None
Est: 200 / Depth: 30 ---- Precision: 1.0 / Recall: 0.705 / Accuracy: 0.959 / f-Score: 0.827 / Support: None
Est: 200 / Depth: None ---- Precision: 1.0 / Recall: 0.827 / Accuracy: 0.976 / f-Score: 0.905 / Support: None
